In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Kiểm tra nếu GPU khả dụng
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model and tokenizer
model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Danh sách câu hỏi mới
questions = [
    "Liệt kê tất cả máy tính xách tay.",
    "Liệt kê tất cả các cửa hàng.",
    "Cho tôi xem tất cả các máy tính xách tay có bộ xử lý Intel.",
    "Laptop nào có RAM 16GB?",
    "Những máy tính xách tay nào có card đồ họa NVIDIA?",
    "Những loại laptop hiện đang có hàng tại TechMall Đà Nẵng Store?",
    "Những cửa hàng nào bán laptop gaming?",
    "Laptop với card RTX có giá bao nhiêu?",
    "Những cửa hàng nào nằm ở Quận 1?",
    "Laptop gaming nào có hơn 3 chiếc trong kho?",
    "Những laptop nào có cả bộ xử lý Intel Core i7-13700HX và card đồ họa RTX?",
    "Những cửa hàng nào cung cấp laptop với mức giảm giá trên 10%?",
    "Hiển thị các laptop có bộ xử lý Intel Core i7 và RAM 32GB.",
    "Những cửa hàng nào bán laptop gaming với giá dưới 75 triệu đồng?",
    "Những laptop nào có số lượng lớn nhất tại mỗi cửa hàng?",
    "Laptop nào có phần trăm giảm giá lớn nhất ở tất cả các cửa hàng?",
    "Hiển thị những cửa hàng có laptop gaming giảm giá 10%.",
    "Những cửa hàng nào có mức giảm giá tốt nhất cho MacBook?",
    "Sắp xếp giá giảm dần tại LaptopPro Giga Mall.",
    "Liệt kê tên và giá của laptop rẻ nhất tại mỗi cửa hàng.",
    "Liệt kê tất cả các cửa hàng ở Hà Nội bán sản phẩm với mức giảm giá lớn hơn 5%.",
    "Tìm laptop có phần trăm giảm giá cao nhất tại mỗi cửa hàng.",
    "Tìm tất cả các cửa hàng hiện có laptop thuộc danh mục Gaming trong kho và đang giảm giá.",
    "Tìm tất cả các cửa hàng bán laptop có giá trong khoảng từ 15 triệu đến 25 triệu đồng.",
    "Tìm cửa hàng và tên laptop có giá sau giảm dưới 30 triệu đồng.",
    "Tìm cửa hàng và sản phẩm có phần trăm giảm giá cao nhất ở mỗi thành phố.",
    "Tìm tên các laptop chỉ được bán tại một cửa hàng duy nhất.",
    "Tìm những cửa hàng bán laptop với CPU Intel và RAM tối thiểu 16GB.",
    "Tìm tất cả các cửa hàng ở Hà Nội bán laptop Ultrabook nhưng không bán laptop Gaming.",
    "Tìm tất cả các cửa hàng ở quận Hai Bà Trưng có bán ít nhất một laptop với SSD từ 512GB trở lên."
]




# Database schema
schema = """
"Laptops" "Laptop_ID" int , "Laptop_name" text , "Type" text , "Price" float , "CPU" text , "GPU" text , "RAM" text , "SSD" text  , "Description" text , foreign_key:  primary key: "Laptop_ID" [SEP]
"Stores" "Store_ID" int , "Store_name" text , "Address" text , "City" text , "District" text , foreign_key:  primary key: "Store_ID" [SEP]
"Store_Laptop" "ID" int , "Store_ID" int from "Stores" "Store_ID" , "Laptop_ID" int from "Laptops" "Laptop_ID" , "Quantity" int , "Discount_Percentage" float , "Last_Updated" text , foreign_key: "Store_ID" "Laptop_ID" from "Stores" "Store_ID" "Laptops" "Laptop_ID" , primary key: "ID"
"""

# Xử lý từng câu hỏi
for question in questions:
    input_text = f"Question: {question} Schema: {schema}"
    model_inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
    outputs = model.generate(**model_inputs, max_length=512)
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    print(f"Question: {question}")
    print(f"SQL Query: {output_text[0]}\n")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Question: List the name and price of the cheapest laptop at each store.
SQL Query: SELECT T2.Laptop_name, T1.Price, T1.Store_ID FROM Stores AS T1 JOIN Laptops AS T2 ON T1.Laptop_ID = T2.Laptop_ID GROUP BY T1.Store_ID

Question: List all stores in Hanoi that sell products with discounts greater than 5%.
SQL Query: SELECT T2.Store_name FROM Stores AS T1 JOIN Store_Laptop AS T2 ON T1.Store_ID = T2.Store_ID WHERE T1.City = "Hanoi" AND T2.Discount_Percentage > 5%

Question: Find the laptop with the highest percentage discount at each store.
SQL Query: SELECT T1.Laptop_id, T2.Discount_Percentage FROM Stores AS T1 JOIN Store_Laptop AS T2 ON T1.Store_ID = T2.Store_ID GROUP BY T2.Store_ID

Question: Find all stores that currently have laptops in the Gaming category in stock and on sale.
SQL Query: SELECT T2.Store_ID FROM Laptops AS T1 JOIN Store_Laptop AS T2 ON T1.Laptop_ID = T2.Laptop_ID WHERE T1.Type = "Gaming" AND T2.Quantity = 1

Question: Find all stores selling laptops priced between 15 m